# Neural Networks

In [1]:
import pandas as pd

In [2]:
y_train = pd.read_feather('../data/processed/y_train.feather')['voted_up'].to_numpy()
X_train = pd.read_feather('../data/processed/X_train_preprocessed.feather').to_numpy()

In [3]:
y_test = pd.read_feather('../data/processed/y_test.feather')['voted_up'].to_numpy()
X_test = pd.read_feather('../data/processed/X_test_preprocessed.feather').to_numpy()

## Baseline Neural Network

In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.layers import Dense, Dropout, Embedding, GlobalMaxPool1D, LSTM
from tensorflow.keras import Sequential

In [33]:
from keras.preprocessing import text, sequence

In [34]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train.flatten().tolist())
tokenized_list_train = tokenizer.texts_to_sequences(X_train.flatten().tolist())
tokenized_list_test = tokenizer.texts_to_sequences(X_test.flatten().tolist())
X_train_tokenized = sequence.pad_sequences(tokenized_list_train, maxlen=256)
X_test_tokenized = sequence.pad_sequences(tokenized_list_test, maxlen=256)

In [36]:
model = Sequential()

# hidden layers
model.add(Embedding(20000, 512))
model.add(LSTM(50, return_sequences=True))
#model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))

# output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
model.fit(X_train_tokenized, y_train, epochs=5, batch_size=64, validation_data=(X_test_tokenized, y_test))
model.evaluate(X_test_tokenized, y_test)

Epoch 1/5
914/914 [==============================] - 540s 591ms/step - loss: 0.4567 - accuracy: 0.8223 - val_loss: 0.4373 - val_accuracy: 0.8300
Epoch 2/5
914/914 [==============================] - 533s 583ms/step - loss: 0.4237 - accuracy: 0.8391 - val_loss: 0.4344 - val_accuracy: 0.8296
Epoch 3/5
914/914 [==============================] - 533s 583ms/step - loss: 0.4071 - accuracy: 0.8469 - val_loss: 0.4413 - val_accuracy: 0.8324
Epoch 4/5
914/914 [==============================] - 528s 578ms/step - loss: 0.4042 - accuracy: 0.8478 - val_loss: 0.4565 - val_accuracy: 0.8316
Epoch 5/5
457/457 [==============================] - 35s 77ms/step - loss: 0.4504 - accuracy: 0.8305


[0.4504031538963318, 0.8305438160896301]

## Hyperparameter Tuning

In [ ]:
from talos import Scan

In [ ]:
def dense_network(x_train, y_train, x_val, y_val, params):
    model = Sequential()

    # hidden layers
    model.add(Dense(params['dense'], input_dim=8000, activation=params['activation1']))
    model.add(Dropout(params['dropout']))
    model.add(Dense(params['dense']*2, activation=params['activation1']))
    model.add(Dropout(params['dropout']))
    model.add(Dense(params['dense']*0.5, activation=params['activation1']))
    model.add(Dropout(params['dropout']))
    model.add(Dense(params['dense']*0.75, activation=params['activation1']))
    # output layer
    model.add(Dense(1, activation=params['activation2']))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    out = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=5,
                    verbose=0)

    return out, model

In [ ]:
params = {'dropout': [0.25, 0.5, 0.75], 
          'dense': [10, 50, 100, 500], 
          'activation1': ['relu', 'elu'], 
          'activation2': ['sigmoid', 'tanh']}

In [ ]:
results = Scan(X_train, y_train, params=params, model=dense_network, experiment_name='grid')
results.best_model(metric='accuracy')

In [ ]:
pd.read_csv('grid/022421165029.csv').sort_values('val_accuracy', ascending=False)

## Final Model

In [ ]:
model = Sequential()

# hidden layers
model.add(Dense(500, input_dim=8000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(250, activation='relu'))

# output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.save('../final_model/model.h5')
model.save_weights('../final_model/model_weights.h5')